# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

- **Consider a significance level of 5% for all tests.**

In [1]:
import pandas as pd 
import numpy as np
from scipy import stats  # stastiques test 

# Challenge 1 - Independent Sample T-tests

In this challenge, we will be using the Pokemon dataset. Before applying statistical methods to this data, let's first examine the data.

To load the data, run the code below.

In [2]:
# Run this code:

pokemon = pd.read_csv('../pokemon.csv')

Let's start off by looking at the `head` function in the cell below.

In [3]:
pokemon.head(5)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


The first thing we would like to do is compare the legendary Pokemon to the regular Pokemon. To do this, we should examine the data further. What is the count of legendary vs. non legendary Pokemons?

In [4]:
# Your code here:
df_pokemon=pokemon.copy() # copy of data
df_pokemon['Legendary'].value_counts()


Legendary
False    735
True      65
Name: count, dtype: int64

Compute the mean and standard deviation of the total points for both legendary and non-legendary Pokemon.

In [5]:
# Your code here:
df_pokemon.groupby('Legendary')['Total'].agg(['mean', 'std'])


,mean,std
Legendary,,
False,417.213605,106.760417
True,637.384615,60.937389


The computation of the mean might give us a clue regarding how the statistical test may turn out; However, it certainly does not prove whether there is a significant difference between the two groups.

In the cell below, use the `ttest_ind` function in `scipy.stats` to compare the the total points for legendary and non-legendary Pokemon. Since we do not have any information about the population, assume the variances are not equal.

In [6]:
#Extract Total for for each group
pokemon_nlegendary = df_pokemon[df_pokemon["Legendary"] == False]['Total'].dropna()
pokemon_legendary = df_pokemon[df_pokemon["Legendary"] == True]['Total'].dropna()

t_stat, p_value = stats.ttest_ind(pokemon_nlegendary, pokemon_legendary, equal_var=False)
print(f"Test Statistic (t): {t_stat:.2f}")
print(f"P-Value: {p_value:.5f}")
print()

# Significance level
alpha = 0.05


Test Statistic (t): -25.83
P-Value: 0.00000



What do you conclude from this test? Write your conclusions below.

In [7]:
# Decision-Making
if p_value > alpha:
    print("Fail to Reject the Null Hypothesis: The mean total for Legendary and non-Legendary are not significantly different.")
else:
    print("Reject the Null Hypothesis: There is sufficient evidence to conclude that the mean Total are different for Legendary and non-Legendary.")


Reject the Null Hypothesis: There is sufficient evidence to conclude that the mean Total are different for Legendary and non-Legendary.


How about we try to compare the different types of pokemon? In the cell below, list the types of Pokemon from column `Type 1` and the count of each type.

In [8]:
Type_1_pokemon= df_pokemon["Type 1"].unique()
print(f"list of Pokemon Type 1 {Type_1_pokemon}")

df_pokemon["Type 1"].value_counts()

list of Pokemon Type 1 ['Grass' 'Fire' 'Water' 'Bug' 'Normal' 'Poison' 'Electric' 'Ground'
 'Fairy' 'Fighting' 'Psychic' 'Rock' 'Ghost' 'Ice' 'Dragon' 'Dark' 'Steel'
 'Flying']


Type 1
Water       112
Normal       98
Grass        70
Bug          69
Psychic      57
Fire         52
Electric     44
Rock         44
Dragon       32
Ground       32
Ghost        32
Dark         31
Poison       28
Steel        27
Fighting     27
Ice          24
Fairy        17
Flying        4
Name: count, dtype: int64


Since water is the largest group of Pokemon, compare the mean and standard deviation of water Pokemon to all other Pokemon.

In [9]:
# create a new colomn with 2 group Water: pokemon type 1= water  and not_water: pokemon type 1!= water
df_pokemon['group_type1'] = ['Water' if t=='Water' else 'non_Water' for t in df_pokemon['Type 1']] 
df_pokemon.head(3)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,group_type1
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False,non_Water
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False,non_Water
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False,non_Water


In [10]:
df_pokemon.groupby('group_type1')['Total'].agg(['mean', 'std','count']).reset_index()

,group_type1,mean,std,count
0,Water,430.455357,113.188266,112
1,non_Water,435.859012,121.091682,688


Perform a hypothesis test comparing the mean of total points for water Pokemon to all non-water Pokemon. Assume the variances are equal. 

In [11]:
non_Water=df_pokemon[df_pokemon['group_type1']=='non_Water']['Total'].dropna()
Water=df_pokemon[df_pokemon['group_type1']=='Water']['Total'].dropna()

# Perform paired t-test
t_stat, p_value = stats.ttest_ind(non_Water, Water, equal_var=False)
print(f"Test Statistic (t): {t_stat:.2f}")
print(f"P-Value: {p_value:.5f}")
print()

# Significance level
alpha = 0.05

Test Statistic (t): 0.46
P-Value: 0.64339



Write your conclusion below.

In [12]:
if p_value > alpha:
    print("Fail to Reject the Null Hypothesis: the mean of total points for water Pokemon and all non-water Pokemon are not significantly different.")
else:
    print("Reject the Null Hypothesis: There is sufficient evidence to conclude that the mean of total points for water Pokemon and all non-water Pokemon are different.")



Fail to Reject the Null Hypothesis: the mean of total points for water Pokemon and all non-water Pokemon are not significantly different.


# Challenge 2 - Matched Pairs Test

In this challenge we will compare dependent samples of data describing our Pokemon. Our goal is to see whether there is a significant difference between each Pokemon's defense and attack scores. Our hypothesis is that the defense and attack scores are equal. In the cell below, import the `ttest_rel` function from `scipy.stats` and compare the two columns to see if there is a statistically significant difference between them.

In [13]:
df_p=pokemon.copy()
df_p.head(2)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False


In [14]:
attack=df_p['Attack']
defense=df_p['Defense']

In [16]:
t_stat, p_value = stats.ttest_rel(attack, defense)
print(f"Test Statistic (t): {t_stat:.2f}")
print(f"P-Value: {p_value:.5f}")
print()



Test Statistic (t): 4.33
P-Value: 0.00002



Describe the results of the test in the cell below.

In [17]:
# Significance level
alpha = 0.05

# Decision-Making
if p_value > alpha:
    print("Fail to Reject the Null Hypothesis: No significant difference between each Pokemon's defense and attack scores.")
else:
    print("Reject the Null Hypothesis: There is a significant difference between each Pokemon's defense and attack scores.")

Reject the Null Hypothesis: There is a significant difference between each Pokemon's defense and attack scores.


We are also curious about whether therer is a significant difference between the mean of special defense and the mean of special attack. Perform the hypothesis test in the cell below. 

In [18]:
sp_attack=df_p['Sp. Atk']
sp_defense=df_p['Sp. Def']

In [19]:
t_stat, p_value = stats.ttest_rel(sp_attack, sp_defense)
print(f"Test Statistic (t): {t_stat:.2f}")
print(f"P-Value: {p_value:.5f}")
print()

Test Statistic (t): 0.85
P-Value: 0.39337



Describe the results of the test in the cell below.

In [20]:
# Significance level
alpha = 0.05

# Decision-Making
if p_value > alpha:
    print("Fail to Reject the Null Hypothesis: No significant difference between the mean of special defense and the mean of special attack.")
else:
    print("Reject the Null Hypothesis: There is a significant difference the mean of special defense and the mean of special attack.")


Fail to Reject the Null Hypothesis: No significant difference between the mean of special defense and the mean of special attack.


As you may recall, a two sample matched pairs test can also be expressed as a one sample test of the difference between the two dependent columns.

Import the `ttest_1samp` function and perform a one sample t-test of the difference between defense and attack. Test the hypothesis that the difference between the means is zero. Confirm that the results of the test are the same.

In [24]:
# Your code here:
df_p['diff'] = [a-b  for a,b in zip( df_p['Sp. Atk'], df_p['Sp. Def'])] 
diff=df_p['diff']
mu=0

In [25]:
df_p['Sp. Atk'].mean() - df_p['Sp. Def'].mean()

0.9174999999999898

In [26]:
t_stat, p_value = stats.ttest_1samp(diff, mu)
# print(f"Test Statistic (t): {t_stat:.2f}")
print(f"P-Value: {p_value:.4f}")
print()
if p_value > alpha:
    print("Fail to Reject the Null Hypothesis: Not enough evidence to say the difference between the means is zero..")
else:
    print("Reject the Null Hypothesis: There is evidence to say the difference between the means is zero.")

P-Value: 0.3934

Fail to Reject the Null Hypothesis: Not enough evidence to say the difference between the means is zero..


# Bonus Challenge - The Chi-Square Test

The Chi-Square test is used to determine whether there is a statistically significant difference in frequencies. In other words, we are testing whether there is a relationship between categorical variables or rather when the variables are independent. This test is an alternative to Fisher's exact test and is used in scenarios where sample sizes are larger. However, with a large enough sample size, both tests produce similar results. Read more about the Chi Squared test [here](https://en.wikipedia.org/wiki/Chi-squared_test).

In the cell below, create a contingency table using `pd.crosstab` comparing whether a Pokemon is legenadary or not and whether the Type 1 of a Pokemon is water or not.

In [27]:
# crosstab Legendary and group_type1 (groupe 1 type of pokemon: water or non water 
crosstab_result = pd.crosstab(df_pokemon['Legendary'], df_pokemon['group_type1'])
crosstab_result

group_type1,Water,non_Water
Legendary,,
False,108,627
True,4,61


Perform a chi-squared test using the `chi2_contingency` function in `scipy.stats`. You can read the documentation of the function [here](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.chi2_contingency.html).

In [28]:
stats.chi2_contingency(crosstab_result)

Chi2ContingencyResult(statistic=2.94292007628505, pvalue=0.08625467249550997, dof=1, expected_freq=array([[102.9, 632.1],
       [  9.1,  55.9]]))

Based on a 95% confidence, should we reject the null hypothesis?